In [ ]:
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, EulerDiscreteScheduler
import torch
from PIL import Image
import os

model_id = "stabilityai/stable-diffusion-2-base"

In [ ]:
# Use the Euler scheduler here instead
scheduler = EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id,
                                               scheduler=scheduler,
                                               revision="fp16",
                                               torch_dtype=torch.float16,
                                               safety_checker=None)
pipe = pipe.to("cuda")

In [ ]:
prompt = "lowres, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, ((((ugly)))), (((duplicate))), ((morbid)), ((mutilated))"
num_samples = 4 

if not os.path.exists("wrong_images"):
    os.mkdir("wrong_images")

i = 0
for cfg in range(7, 20+1):
    with torch.autocast("cuda"):
        images = pipe(prompt,
                      negative_prompt=None,
                      num_images_per_prompt=num_samples,
                      num_inference_steps=50, guidance_scale=cfg).images

        for image in images:
            image.save(os.path.join("wrong_images", f"{i:03d}.png"))
            i += 1
        